In [161]:
from importlib import reload
reload(change)
reload(base)

<module 'src.base' from 'C:\\Users\\ugur.eren\\Python Codes\\cefis2\\src\\base.py'>

In [162]:
import pandas as pd
import numpy as np
import src.in_out as io
import src.info as info
import src.auxiliary_functions as aux
import src.condition as con
import src.base as base
from itertools import combinations
from src.spread import get_spread
import src.change as change

## Veri Hazırlama

In [2]:
#Verilere ait path'ler alınıyor
path_list=io.get_path('data')

In [3]:
#Okunacak sütun adları
cols= ["Time","BID price","ASK price"]
#Tüm veriler okunuyor
master_data=io.read_allData(path_list, cols=cols)
master_data.columns=['time','bid_price','ask_price','name','date']

In [4]:
#date sütunu Timestamp'e dönüştürülüyor ve tarih ve saat bilgisi birleştiriliyor
master_data['date']=pd.to_datetime(master_data['date'], errors='coerce')
master_data['date'] += pd.to_timedelta(master_data.pop('time').astype(str))

In [5]:
#zaman dilimi ekleniyor
master_data['time_period']=master_data.date.dt.hour

## Mid Price

In [6]:
# mid price hesaplanıyor
master_data['mid_price']=base.average_of_series(master_data.bid_price,master_data.ask_price)

In [7]:
# spread hesaplama için master_data, uygun bir forma dönüştürülüyor
df_mid_price=master_data.pivot(index='date', columns='name', values='mid_price')

## Spread
<img src="img/spread_flowchart.png">
<br/>
Spread, her bir saat için ayrı ayrı hesaplanmıştır

In [8]:
values={'a_PNLTICK':1,
'a_TICKSIZE':1,
'b_PNLTICK':1,
'b_TICKSIZE':1}

In [9]:
# pair'lar alınıyor
pairs=combinations(df_mid_price.columns, 2)

In [10]:
#spread hesaplanıyor
spread_list=[
    base.groupby_date_time(df_mid_price.loc[:,pair]).apply(lambda x: get_spread(x,values))\
        for pair in pairs]
df_spread=pd.concat(spread_list,axis=1).droplevel([0,1])

## Change

In [64]:
#change alınıyor
df_change=base.groupby_date_time(df_spread).apply(change.get_change)
df_change=df_change.droplevel([0,1])

## Amplitude

In [170]:
all_pairs_of_amplitude=[]

In [182]:
for pair_change in df_change:   #Her bir pair alınıyor
    pair_group = list(base.groupby_date_time(df_change[pair_change])) #gün ve saate göre gruplanıyor
    sub_amplitudes=[]
    for i in range(len(pair_group)):
        sub_change=base.to_series(pair_group[i][1])
        sub_amplitudes.append(change.get_amplitude(sub_change)) #pair'a ait her alt grubun amplitudu'u hesaplanıyor
    all_pairs_of_amplitude.append(pd.concat(sub_amplitudes)) # alt amplitude'lar birleştiriliyor

In [190]:
from functools import reduce

In [200]:
# Amplitude'u hesaplanan tüm pair'lar birleştiriliyor.
df_final = reduce(lambda left,right: pd.merge(left,right,on=['date','duration'],how='outer'), all_pairs_of_amplitude)

In [234]:
df_final # NaN'lar birleştirmeden dolayı. ÖNEMSİZ ! 6AU8_6CU8, 2018-09-06 00:00:49 dan başladığından öncesi NaN

,6AU8_6BU8,duration,date,6AU8_6CU8,6BU8_6CU8
0,-0.000218,00:00:13,2018-09-06 00:00:17,NaN,0.000218
1,0.000050,00:00:22,2018-09-06 00:00:39,NaN,-0.000050
2,-0.000059,00:00:09,2018-09-06 00:00:48,NaN,0.000059
3,0.000047,00:00:01,2018-09-06 00:00:49,-0.000003,-0.000050
4,-0.000047,00:00:01,2018-09-06 00:00:50,0.000003,0.000050
...,...,...,...,...,...
11864,NaN,00:00:05,2018-09-07 23:51:36,NaN,0.000050
11865,NaN,00:00:01,2018-09-07 23:55:36,NaN,0.000050
11866,NaN,00:00:17,2018-09-07 23:55:54,NaN,0.000100
11867,NaN,00:00:02,2018-09-07 23:58:52,NaN,-0.000100


# Sonuçlar Excele Yazdırılıyor

In [235]:
df_mid_price.to_excel('out/mid_price.xlsx')
df_spread.to_excel('out/spread.xlsx')
df_change.to_excel('out/change.xlsx')
df_final.to_excel('out/amplitude.xlsx')